Hierin staan alle modellen die gebruikt worden in de lecture notes.

<h1><center> Lecture notes</center></h1>


# A. KIVA model

In [2]:
import math
from PyCh import *

PyCh version 0.1 imported succesfully.
 


In [2]:
from PyCh import *

# =================================
# Pod definition
# =================================
@dataclass
class Pod:
    id: int = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, a, N):
    for i in range(N):
        x = Pod()
        send = a.send(x)
        yield env.execute(send)

# =================================
# Storage definition
# =================================
@process
def Storage(env, a, b, la):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)

        delay = random.exponential(1.0 / la)
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, a, b):
    xs = [] # list of pods
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]


# =================================
# Pick definition
# =================================
@process
def Pick(env, a, b, mu, n):
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)

        delay = random.exponential(1.0 / mu)
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)
    print("TH = %g" % ( n / env.now))

# =================================
# KIVA Model
# =================================
def model():
    N = 1       # Number of robots
    la = 4.0    # average rate / hour for storing and retrieving racks (averages to 15 min per task)
    mu = 20.0   # average rate / hour for picking of racks (averages to 3 min per task)

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, N)
    Ss = [Storage(env, a, b, la) for j in range(N)]
    B = Buffer(env, b, c)
    P = Pick(env, c, a, mu, 10000)

    env.run()
    
# =================================
# Main
# =================================
model()

TH = 3.35552
simulation has ended



# B. Zero-buffer model

In [3]:
from PyCh import *

# =================================
# Job definition
# =================================
@dataclass
class Job:
    id: int = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, a, u):
    i = 0
    while True:
        x = Job(id = i)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)
        i = i+1

# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)

        delay = u()
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)

# =================================
# Exit definition
# =================================
@process
def Exit(env, a, n):
    x = Job()
    while x.id < n:
        receive = a.receive()
        x = yield env.execute(receive)
    print("TH = %g" % ( x.id / env.now))

# =================================
# GME Model
# =================================
def model():
    ta = 1.0
    te = 1.0
    n  = 1000

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    G = Generator(env, a, lambda: ta)
    M = Machine(env, a, b, lambda: te)
    E = Exit(env, b, n)

    env.run()

# =================================
# Main
# =================================
model()

TH = 0.999001
simulation has ended


# C. Finite Buffer Model

In [11]:
from PyCh import *

# =================================
# Job definition
# =================================
@dataclass
class Job:
    id: int = 0

# =================================
# Generator definition
# =================================
@process
def Generator(env, a, u):
    i = 0
    while True:
        x = Job(id = i)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)
        i = i+1

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, a, b, N):
    xs = []
    while True:
        receiving = a.receive() if len(xs)<N else None
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)

        delay = u()
        yield env.timeout(delay)

        send = b.send(x)
        yield env.execute(send)

# =================================
# Exit definition
# =================================
@process
def Exit(env, a, n):
    x = Job()
    while x.id < n:
        receive = a.receive()
        x = yield env.execute(receive)
    print("TH = %g" % ( x.id / env.now))

# =================================
# GBME Model
# =================================
def model():
    ta = 1.0
    te = 1.0
    n  = 1000
    N = 10

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, lambda: random.exponential(ta))
    B = Buffer(env, a, b,  N)
    M = Machine(env, b, c, lambda: random.exponential(te))
    E = Exit(env, c, n)

    env.run()
    
# =================================
# Main
# =================================
model()

TH = 0.90992


# D. Single Machine Model

In [6]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# Generator definition
# =================================
@process
def Generator(env, a, u):
    while True:
        x = Job(entrytime = env.now)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, a, b):
    xs = []
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# Exit definition
# =================================
@process
def Exit(env, a, n):
    sumw = 0.0
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    print("Mean waiting time spent in buffer = %g" % (sumw / n))

# =================================
# GBME Model
# =================================
def model():
    n  = 100000

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, lambda: random.uniform(0.0, 2.0))
    B = Buffer(env, a, b)
    M = Machine(env, b, c, lambda: random.uniform(0.0, 1.0))
    E = Exit(env, c, n)

    env.run(until=E)
    
# =================================
# Main
# =================================
model()

Mean waiting time spent in buffer = 0.149063
simulation has ended


# E. Multi machine model

In [3]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# Generator definition
# =================================
@process
def Generator(env, a, u):
    while True:
        x = Job(entrytime = env.now)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, a, b):
    xs = []
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# Exit definition
# =================================
@process
def Exit(env, a, n):
    sumw = 0.0
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    print("Mean waiting time spent in buffer = %g" % (sumw / n))

# =================================
# GBME Model
# =================================
def model():
    m = 2
    n = 1000000

    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, lambda: random.exponential(0.111))
    B = Buffer(env, a, b)
    Ms = [ Machine(env, b, c, lambda: random.uniform(0.0, 0.4)) for j in range(m) ]
    E = Exit(env, c, n)

    env.run(until=E)

# =================================
# Main
# =================================
model()

KeyboardInterrupt: 

# F. Serial production line

In [12]:
# =================================
# Job definition
# =================================
@dataclass
class Job:
    entrytime: float = 0.0

# =================================
# Generator definition
# =================================
@process
def Generator(env, a, u):
    while True:
        x = Job(entrytime = env.now)
        send = a.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Buffer definition
# =================================
@process
def Buffer(env, a, b):
    xs = []
    while True:
        receiving = a.receive()
        sending = b.send(xs[0]) if len(xs)>0 else None
        yield env.select(sending,receiving)
        if selected(sending):
            xs = xs[1:]
        if selected(receiving):
            x = receiving.entity
            xs = xs + [x]       
        
# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)

        delay = u()
        yield env.timeout(delay)


# =================================
# Exit definition
# =================================
@process
def Exit(env, a, n):
    sumw = 0.0
    for i in range(n):
        receive = a.receive()
        x = yield env.execute(receive)
        sumw = sumw + (env.now - x.entrytime)
    return sumw / n

# =================================
# Mline Model
# =================================
def model():
    n = 100000

    env = Environment()
    a = [ Channel(env) ] * 3
    b = [ Channel(env) ] * 2
    G = Generator(env, a[0], lambda: random.exponential(0.5))
    B1 = Buffer(env, a[0], b[0])
    M1 = Machine(env, b[0], a[1], lambda: 0.33)
    B1 = Buffer(env, a[1], b[1])
    M1 = Machine(env, b[1], a[2], lambda: random.uniform(0.0, 0.8))
    E = Exit(env, a[2], n)
    env.run(until=E)
    
    return E.value


# =================================
# Experiment
# =================================
def Experiment():
    m = 10
    phi = 0.0
    sum1 = 0.0
    sum2 = 0.0
    smean = 0.0
    svar = 0.0
    
    for i in range(m):
        phi = model()
        sum1 = sum1 + phi
        sum2 = sum2 + phi*phi
        print("Mean flow time in run %d is %g" %(i+1, phi))
        
    smean = sum1 / m
    svar = sum2 / m - smean * smean
    print("Mean flow time estimate is %g +- %g" % (smean, 1.96 * math.sqrt(svar/m) ))

# =================================
# Main
# =================================
Experiment()

Mean flow time in run 1 is 0.241385
Mean flow time in run 2 is 0.251163
Mean flow time in run 3 is 0.179717
Mean flow time in run 4 is 0.247519
Mean flow time in run 5 is 0.231021
Mean flow time in run 6 is 0.274336
Mean flow time in run 7 is 0.299099
Mean flow time in run 8 is 0.211391
Mean flow time in run 9 is 0.254874
Mean flow time in run 10 is 0.289996
Mean flow time estimate is 0.24805 +- 0.0209826
simulation has ended


In [ ]:
# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        receive = a.receive()
        x = yield env.execute(receive)
        
        send = b.send(x)
        yield env.execute(send)
        
        delay = u()
        yield env.timeout(delay)

# =================================
# Machine definition
# =================================
@process
def Machine(env, a, b, u):
    while True:
        x = yield env.execute(a.receive())
        yield env.execute(b.send(x))
        yield env.timeout(u())


############################################################